In [1]:
import os
os.chdir('../')
import pandas as pd
import numpy as np

# ML 모델 학습

지금까지 배웠던 모델들을 활용해서 이전의 `EDA` 파일에서 만들었던 정제된 데이터셋으로 학습 및 평가를 진행하면 됩니다.     
집값 예측 데이터셋은 회귀 문제이므로, 회귀 모델들을 사용하면 됩니다.     

Ensemble 자료에서 배웠던 모델들은 각각 `Classifier`에서 `Regressor`로 이름을 바꾸면, 회귀 모델이 되는 점 참고해주세요.

In [2]:
X = pd.read_csv('./data/preprocessed_X.csv')
y = pd.read_csv('./data/preprocessed_y.csv')

print("X dataset size:", X.shape)
print("y dataset size:", y.shape)

X dataset size: (1457, 220)
y dataset size: (1457, 1)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

원하는 모델 구현해서 평가 데이터셋에 대한 성능을 찍고 분석해보세요.    
이 때, 회귀 모델에 맞는 평가 지표를 `sklearn.metrics`에서 찾아서 사용해보세요.

In [4]:
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor(penalty='l2', 
                   alpha=0.01,
                   max_iter=1000,
                   tol=1e-3,
                   learning_rate='invscaling',
                   eta0=0.001)

reg.fit(X_train, y_train)

/home/sangmin/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SGDRegressor(alpha=0.01, eta0=0.001)

In [5]:
from sklearn.metrics import mean_squared_error

y_pred = reg.predict(X_test)
loss = mean_squared_error(y_pred, y_test)
print('Linear Regression model loss : %.4f' % loss)

Linear Regression model loss : 8758923202264192661020007727104.0000


In [6]:
from sklearn.svm import SVR

svr = SVR(kernel='rbf', gamma=5, C=5)

svr.fit(X_train, y_train)

/home/sangmin/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVR(C=5, gamma=5)

In [7]:
y_pred = svr.predict(X_test)
loss = mean_squared_error(y_pred, y_test)
print('Support Vector Regression model loss : %.4f' % loss)

Support Vector Regression model loss : 0.1541


In [8]:
from lightgbm import LGBMRegressor

lightgbm = LGBMRegressor(objective='regression', 
                         num_leaves=4,
                         learning_rate=0.01, 
                         n_estimators=5000,
                         max_bin=200, 
                         bagging_fraction=0.75,
                         bagging_freq=5, 
                         bagging_seed=7,
                         feature_fraction=0.2,
                         feature_fraction_seed=7,
                         verbose=-1,
                         #min_data_in_leaf=2,
                         #min_sum_hessian_in_leaf=11
                         )

lightgbm.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.75, subsample=1.0 will be ignored. Current value: bagging_fraction=0.75


LGBMRegressor(bagging_fraction=0.75, bagging_freq=5, bagging_seed=7,
              feature_fraction=0.2, feature_fraction_seed=7, learning_rate=0.01,
              max_bin=200, n_estimators=5000, num_leaves=4,
              objective='regression', verbose=-1)

In [9]:
y_pred = lightgbm.predict(X_test)
loss = mean_squared_error(y_pred, y_test)
print('LightGBM model loss : %.4f' % loss)

LightGBM model loss : 0.0119


In [10]:
from xgboost import XGBRegressor

xgboost = XGBRegressor(learning_rate=0.01, 
                       n_estimators=3460,
                       max_depth=3, 
                       min_child_weight=0,
                       gamma=0, 
                       subsample=0.7,
                       colsample_bytree=0.7,
                       scale_pos_weight=1, 
                       seed=27,
                       reg_alpha=0.00006)
                               

xgboost.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=0, missing=nan,
             monotone_constraints='()', n_estimators=3460, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=27,
             reg_alpha=6e-05, reg_lambda=1, scale_pos_weight=1, seed=27,
             subsample=0.7, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [11]:
y_pred = xgboost.predict(X_test)
loss = mean_squared_error(y_pred, y_test)
print('XGBoost model loss : %.4f' % loss)

XGBoost model loss : 0.0111


### Q1. 어떤 데이터 전처리 과정을 진행했는지 정리해주세요.

손실 데이터셋 제거, 아웃라이어 제거, 필요없는 피처 제거하거나 새로운 변수 생성, 몇몇 변수들의 값 정규화 등의 전처리들을 적용함.     
또한 SalePrice 또한 로그 함수를 취해 값을 변화시킴.

### Q2. 데이터 전처리릍 통해 데이터의 샘플 개수와 피처 차원은 어떻게 바뀌었나요?

처음 데이터셋의 크기는 (1460, 80)의 크기 였음(SalePrice 타겟 피처 고려 없이).     
전처리를 통해 데이터셋의 모양은 (1457, 220)으로 바뀜.     
학습에 방해가 될 샘플 3개를 제거하고, 범주형 데이터를 처리하기 위해 dummy 피처로 변화시키는 과정에서 220개의 피처 개수로 증가함.

### Q3. 어떤 모델을 사용해서 데이터를 학습했으며, 모델의 장점과 단점을 설명해주세요.

Ensemble 모델들 중 최신 모델인 LightGBM과 XGBoost 모델을 사용했고 꽤 좋은 성능을 얻음.     
Boosting 알고리즘들로, 결정 트리 기반으로 구현되었음에도 꽤 좋은 성능들을 보여주는 알고리즘임. 또한, 학습 속도 개선과 ensemble의 효과를 높이고자 한 기술들이 추가된 모델들임.